**Date :** Created on Thursday January 7 2021

**Group 8 - Innovation**

**CosSimilarityByFastText_v0** 

**@author :** Melina Audiger. 

**Description :** Word vectors (by FastText) are used  to calculate a vector for each article in order to calculate a cosinus similarity between two articles. The save_art_similar function returns a Dataframe with the 10 most similar articles for each article with the corresponding score

# Part 1 : Install / Download / Import Librairy

## Download librairy :

In [17]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Import librairy

### - Usefull librairy :

In [18]:
import pandas as pd
import numpy as np
import operator
from google.colab import drive
from tqdm import trange

### - Text librairy :

In [19]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

# Part 2 : Creation of coordinates dictionary

In [20]:
# Connect the drive folder
drive.mount('/content/drive')

# My file path for the data
Helper_path : str = '/content/drive/MyDrive/data_interpromo/G8 Innovation/cc.fr.300.vec'

# My initialized dictionary
Dico : dict = {}

# Open data file
with open(Helper_path, encoding = "utf8") as file:

    # Step 1 : dictionary creation
    for line in file:

        # Split each rows
        Values = line.split(' ')

        # Save Coordinates of each word
        Dico[Values[0]] = np.array(Values[1:]).astype('float')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Part 3 : Data Loading

In [21]:
def Load_data(helper_path : str) -> pd.DataFrame :
    """Documentation
    
    Parameters :
        - helper_path : the file path

    Output (if exists) :
        - df : My Dataframe cleaned and reindexed

    """
    
    # Data Load with pandas librairy
    df = pd.read_csv(Helper_path)

    # Drop articles with no content
    df = df[df['art_content'] != '']

    # Reset my dataframe index
    df = df.reset_index(drop = True)
    
    # Returns my clean dataframe
    return df

In [22]:
# My file path for the fonction
Helper_path : str = '/content/drive/MyDrive/data_interpromo/Data/abstract_v1.csv'

# My DataFrame variable
My_data : pd.DataFrame = Load_data(Helper_path)

# To show my DataFrame
My_data.head(10)

,art_id,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag,art_clean,abstract_sentence,abstract_words
0,1,le FNCDG et l’ andcdg avoir publier en septemb...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,fr,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/09/im...,NaN,NaN,fncdg andcdg avoir publier septembre 9em editi...,fncdg andcdg avoir publier septembre 9em editi...,fncdg andcdg avoir publier septembre 9em editi...
1,2,malgré le levée un mesure de confinement le 11...,"<p style=""text-align: justify;"">Malgré la levé...",17 mars 2020,fr,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/03/co...,NaN,NaN,malgre levee mesure confinement 11 mai 2020 pl...,malgre levee mesure confinement 11 mai 2020 pl...,malgre levee mesure confinement 11 mai 2020 pl...
2,25,quel être le objectif poursuivre par le gouver...,"<p style=""text-align: justify;""><strong>Quels ...",24 octobre 2019,fr,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2019/10/in...,NaN,NaN,quel etre objectif poursuivre gouvernement cad...,quel etre objectif poursuivre gouvernement cad...,quel etre objectif poursuivre gouvernement cad...
3,27,"le journée thématique , qui avoir lieu durant ...","<p style=""text-align: justify;""><strong>La jo...",31 mai 2017,fr,Journée Thématique FNCDG « Les services de san...,http://fncdg.com/journee-thematique-fncdg-les-...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/05/pu...,NaN,NaN,journee thematique avoir lieu durant salon pre...,journee thematique avoir lieu durant salon pre...,journee thematique avoir lieu durant salon pre...
4,28,le 1ère journée thématique en région sur le th...,"<p style=""text-align: justify;"">La 1<sup>ère</...",13 mars 2017,fr,Journée Thématique FNCDG « Vers de nouveaux mo...,http://fncdg.com/journee-thematique-fncdg-vers...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/03/Sa...,NaN,NaN,1ere journee thematique region theme vers nouv...,1ere journee thematique region theme vers nouv...,1ere journee thematique region theme vers nouv...
5,30,l’ un un innovation de le loi n degré 2019 - 8...,"<p style=""text-align: justify;"">L’une des inno...",22 octobre 2020,fr,La publication d’un guide d’accompagnement à l...,http://fncdg.com/la-publication-dun-guide-dacc...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/10/LG...,NaN,NaN,innovation loi degre 2019 828 6 aout 2019 dire...,innovation loi degre 2019 828 6 aout 2019 dire...,innovation loi degre 2019 828 6 aout 2019 dire...
6,31,"le FNCDG mener , en collaboration avec d’ autr...","<p style=""text-align: justify;"">La FNCDG mène,...",10 décembre 2020,fr,La publication d’un guide de sensibilisation a...,http://fncdg.com/la-publication-dun-guide-de-s...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/12/im...,NaN,NaN,fncdg mener collaboration autre partenaire cam...,fncdg mener collaboration autre partenaire cam...,fncdg mener collaboration autre partenaire cam...
7,32,"créer pour et par le décideur territorial , ét...","<p style=""text-align: justify;"">Créé pour et p...",24 février 2017,fr,Lancement du réseau Étoile,http://fncdg.com/lancement-du-reseau-etoile/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/02/re...,NaN,NaN,creer decideur territorial etoile etre tout pr...,creer decideur territorial etoile etre tout pr...,creer decideur territorial etoile etre tout pr...
8,34,le décret n

# Part 4 : Similarity calculations

## - Phase 1 : Create the initials coordinates

In [23]:
def Coord_initial(df : pd.DataFrame, column : str) -> pd.DataFrame :
    """Documentation
    
    Parameters :
        - df : My Dataframe for create coordinates

    Output (if exists) :
        - df : My Dataframe with the coordinates

    """

    # My initialized list
    list_coord : list = []

    # Step 1 : calculate coordinates
    for i in trange(len(df) , position = 0 , leave = True):
    
        # Get the item
        item = df[column][i]

        # Convert all object to str()
        if not isinstance(item, str):
  
            item = str(item)

        # My initialized coordinates
        coord_art : list = [0]*300

        # Step 2 : Tokenize all word
        for word in nltk.word_tokenize(item):
      
            # Get the word
            try:
          
                # Add the coordinates of word
                coord_art += Dico[word]
      
            except: # get initial word

                word

        # Save coordinates
        list_coord.append(coord_art)

    # Return my coordinates list 
    return list_coord 

In [24]:
# My initialized column 
My_data['art_coor'] : pd.DataFrame = ''

# Application of my function
My_data['art_coor'] = Coord_initial(My_data, 'art_clean')

# To show my DataFrame
My_data.head(10)

100%|██████████| 7490/7490 [00:30<00:00, 245.57it/s]


,art_id,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag,art_clean,abstract_sentence,abstract_words,art_coor
0,1,le FNCDG et l’ andcdg avoir publier en septemb...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,fr,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/09/im...,NaN,NaN,fncdg andcdg avoir publier septembre 9em editi...,fncdg andcdg avoir publier septembre 9em editi...,fncdg andcdg avoir publier septembre 9em editi...,"[-0.0023, -0.24740000000000015, -0.586, 0.0272..."
1,2,malgré le levée un mesure de confinement le 11...,"<p style=""text-align: justify;"">Malgré la levé...",17 mars 2020,fr,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/03/co...,NaN,NaN,malgre levee mesure confinement 11 mai 2020 pl...,malgre levee mesure confinement 11 mai 2020 pl...,malgre levee mesure confinement 11 mai 2020 pl...,"[0.7350999999999998, 0.8879999999999999, -2.97..."
2,25,quel être le objectif poursuivre par le gouver...,"<p style=""text-align: justify;""><strong>Quels ...",24 octobre 2019,fr,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2019/10/in...,NaN,NaN,quel etre objectif poursuivre gouvernement cad...,quel etre objectif poursuivre gouvernement cad...,quel etre objectif poursuivre gouvernement cad...,"[6.953399999999994, 5.824000000000012, -20.629..."
3,27,"le journée thématique , qui avoir lieu durant ...","<p style=""text-align: justify;""><strong>La jo...",31 mai 2017,fr,Journée Thématique FNCDG « Les services de san...,http://fncdg.com/journee-thematique-fncdg-les-...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/05/pu...,NaN,NaN,journee thematique avoir lieu durant salon pre...,journee thematique avoir lieu durant salon pre...,journee thematique avoir lieu durant salon pre...,"[0.5617000000000001, 0.7993000000000001, -1.63..."
4,28,le 1ère journée thématique en région sur le th...,"<p style=""text-align: justify;"">La 1<sup>ère</...",13 mars 2017,fr,Journée Thématique FNCDG « Vers de nouveaux mo...,http://fncdg.com/journee-thematique-fncdg-vers...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/03/Sa...,NaN,NaN,1ere journee thematique region theme vers nouv...,1ere journee thematique region theme vers nouv...,1ere journee thematique region theme vers nouv...,"[0.9700000000000001, 0.5553000000000001, -3.56..."
5,30,l’ un un innovation de le loi n degré 2019 - 8...,"<p style=""text-align: justify;"">L’une des inno...",22 octobre 2020,fr,La publication d’un guide d’accompagnement à l...,http://fncdg.com/la-publication-dun-guide-dacc...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/10/LG...,NaN,NaN,innovation loi degre 2019 828 6 aout 2019 dire...,innovation loi degre 2019 828 6 aout 2019 dire...,innovation loi degre 2019 828 6 aout 2019 dire...,"[1.3232999999999997, 1.643100000000001, -3.664..."
6,31,"le FNCDG mener , en collaboration avec d’ autr...","<p style=""text-align: justify;"">La FNCDG mène,...",10 décembre 2020,fr,La publication d’un guide de sensibilisation a...,http://fncdg.com/la-publication-dun-guide-de-s...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/12/im...,NaN,NaN,fncdg mener collaboration autre partenaire cam...,fncdg mener collaboration autre partenaire cam...,fncdg mener collaboration autre partenaire cam...,"[1.3113000000000006, 1.2941, -2.51720000000000..."
7,32,"créer pour et par le décideur territorial , ét...","<p style=""text-align: justify;"">Créé pour

## - Phase 2 : Calculate the similarity :

In [26]:
def Similarity(art_coor1 : list, art_coor2 : list) -> float :
    """Documentation
    
    Parameters :
        - art_coor1 : list of coordinates of article 1
        - art_coor2 : list of coordinates of article 2

    Output (if exists) :
        - similarity : similarity of articles

    """

    # My initialized similarity
    similarity  : int = 0

    # Get all items
    try:
      
        # Calculate similarity
        similarity : float = float(np.dot(art_coor1,art_coor2.T)/ \
                         (np.linalg.norm(art_coor1)*np.linalg.norm(art_coor2)))
    
    except: #if any similarity

        similarity : int = 0
    
    #return similarity
    return similarity

## - Phase 3 : Save articles similar :

In [46]:
def Save_similar(df_id, df_coor, nb_similar : int) -> pd.DataFrame :
    """Documentation
    
    Parameters :
        - df_id : identification of article
        - df_coor : coordinates of article
        - nb_similar : number of article similar to save

    Output (if exists) :
        - similarity : list of article similar for each article

    """
    
    # My personnal list
    df_columns : list = ['art_id','art_sim','score']

    # My initialized DataFrame
    df_art_similar : pd.DataFrame = pd.DataFrame(columns = df_columns)

    # Step 1 : Browse all the article
    for i in trange(len(My_data) , position = 0 , leave = True):
      
        # My initialized list
        list_similar : list = []

        # Step 2 : Browse other article for an article
        for j in range(len(df_id)):
        
            # Check difference 
            if i != j:

                # My first item 
                item_1 : list = df_coor[i]

                # My second item
                item_2 : list = df_coor[j]

                # Calcul of similarity
                simil = Similarity(item_1, item_2)

                # Retrieving the identifier
                df_id_j : int = df_id[j]
          
                # Save the similarity
                list_similar.append([df_id_j, simil])

        # Check if too many similar items
        if len(list_similar) > nb_similar:
        
            # Collapse the list
            list_similar = sorted(list_similar, \
                                  key = operator.itemgetter(1), \
                                  reverse = True)[0:nb_similar]
      
        # Step 3 : Save similar articles and score 
        for listing in list_similar:

            # Retrieving the identifier
            df_id_i : int = df_id[i]

            # Retrieving similarities list
            sim_list : int = listing[0]
      
            # Retrieving score
            sim_score : float = listing[1]
      
            # List creation
            df_art_similar = df_art_similar.append({'art_id' : df_id_i, \
                                                    'art_sim' : sim_list, \
                                                    'score' : sim_score}, \
                                                    ignore_index = True)
    
    # Delete infinity values
    df_art_similar = df_art_similar.drop(df_art_similar[df_art_similar['art_id'] == np.inf].index)

    # Change of type 
    df_art_similar['art_id'] = df_art_similar['art_id'].astype('long')
    df_art_similar['art_sim'] = df_art_similar['art_sim'].astype('long')

    # Return my similarity DataFrame
    return df_art_similar

In [50]:
# Fonction Application
Df_art_simil = Save_similar(My_data['art_id'], My_data['art_coor'], 10)

# To show my DataFrame
Df_art_simil.head(30)

100%|██████████| 7490/7490 [29:32<00:00,  4.23it/s]


,art_id,art_sim,score
0,1,28,0.926145
1,1,6728,0.922189
2,1,9710,0.921121
3,1,11726,0.919081
4,1,9712,0.919027
5,1,9704,0.916143
6,1,6997,0.916022
7,1,8844,0.915933
8,1,1192,0.915728
9,1,9668,0.915453


# Part 6 : Export data

In [51]:
Df_art_simil.to_csv('article_similar.csv', index = False, encoding = 'utf-8')